In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt

torch.cuda.is_available()

True

In [2]:
batch_size = 4
learning_rate = 0.001
epoch = 50
num_category = 1

num_mode =3 # t1,t2,flair
img_size = 32
is_multi_gpu = False

In [3]:
resnet = models.resnet50(pretrained=False)
inception = models.inception_v3(pretrained=False)

#list(resnet.children())[:]

In [4]:
class Resnet(nn.Module):
    def __init__(self):
        super(Resnet,self).__init__()
        self.layer0 = nn.Conv2d(num_mode*3,64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.layer1 = nn.Sequential(*list(resnet.children())[1:-2])        
        
    def forward(self,x):
        out = self.layer0(x)
        out = self.layer1(out)
        #out = out.view(batch_size,-1)
        return out

In [5]:
class Inception(nn.Module):
    def __init__(self):
        super(Inception,self).__init__()
        self.layer0 = nn.Sequential(
            nn.Conv2d(num_mode*3,32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True)
        )
        self.layer1 = nn.Sequential(*list(inception.children())[1:-1])        
        
    def forward(self,x):
        out = self.layer0(x)
        out = self.layer1(out)
        #out = out.view(batch_size,-1)
        return out

In [6]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier,self).__init__()
        self.layer0 = nn.Sequential(
            nn.Conv2d(4096,100,kernel_size=1,bias=False),
            nn.BatchNorm2d(100),
            nn.ReLU(inplace=True),
            nn.Conv2d(100,1,kernel_size=1,bias=False),
            nn.ReLU(inplace=True)
        )
        
    def forward(self,x):
        out = self.layer0(x)
        out = out.view(batch_size,-1)
        return out
    

In [7]:
if torch.cuda.is_available and is_multi_gpu:
    feature_extractor_1 = nn.DataParallel(Resnet()).cuda()
    feature_extractor_2 = nn.DataParallel(Resnet()).cuda()
    classifier = nn.DataParallel(Classifier()).cuda()
    
elif torch.cuda.is_available:
    feature_extractor_1 = Resnet().cuda()
    feature_extractor_2 = Resnet().cuda()
    classifier = Classifier().cuda()
    
else:
    feature_extractor_1 = Resnet()
    feature_extractor_2 = Resnet()
    classifier = Classifier()

In [8]:
if torch.cuda.is_available:
    x1 = Variable(torch.Tensor(batch_size,num_mode*3,img_size,img_size)).cuda()
    x2 = Variable(torch.Tensor(batch_size,num_mode*3,img_size,img_size)).cuda()
else:
    x1 = Variable(torch.Tensor(batch_size,num_mode*3,img_size,img_size))
    x2 = Variable(torch.Tensor(batch_size,num_mode*3,img_size,img_size))

In [9]:
out_1= feature_extractor_1(x1)
out_2= feature_extractor_2(x2)

concat_out = torch.cat([out_1,out_2],dim=1)

print(concat_out.size())

out = classifier(concat_out)
print(out.size())

torch.Size([4, 4096, 1, 1])
torch.Size([4, 1])
